In [29]:
# Testing Tensorflow 2.0 APIs with the builtin fashion mnist dataset.
# refer to: https://zhuanlan.zhihu.com/p/70232196

import numpy as np
import tensorflow as tf


fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [30]:
# make the input dataset 4-D
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))

# standardise ([0, 1] range)
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')

# make tf dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)


# model
class FMModel(tf.keras.Sequential):
    def __init__(self):
        super(FMModel, self).__init__([
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(64, 3, activation='relu'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(10, activation=None)
        ])

model = FMModel()


# decayed_learning_rate = initial_learining_rate * decay_rate^floor(global_step / decay_steps),
# use floor value when staircase=True.
initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

# optimizer
# RMSprop is kind of discounted moving average gradient, an improved version of tf.keras.optimizers.Adagrad.
optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)


# checkpoint
checkpoint = tf.train.Checkpoint(step=tf.Variable(0), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(checkpoint, './tf_ckpts', max_to_keep=3)


# loss function
# sparse categorical cross entropy explain: https://tensorflow.google.cn/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits?hl=en
# logits explain: https://stackoverflow.com/questions/34240703/what-is-logits-softmax-and-softmax-cross-entropy-with-logits?noredirect=1&lq=1%5D
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


# metrics
train_loss_metric = tf.keras.metrics.Mean(name='train_loss')
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss_metric = tf.keras.metrics.Mean(name='test_loss')
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [31]:
# train loop

@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(targets, predictions)
        loss += sum(model.losses)
    # compute gradients and update variables
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss_metric(loss)
    train_acc_metric(targets, predictions)


epochs = 10
for epoch in range(epochs):
    print(f'Start of epoch {epoch}')
    # iterate over the batches of input dataset
    for step, (inputs, targets) in enumerate(train_ds):
        train_step(inputs, targets)
        checkpoint.step.assign_add(1)
        # log every 20 steps
        if step % 20 == 0:
            manager.save()
            print(f'Epoch: {epoch}, Step: {step}, Train Loss: {train_loss_metric.result().numpy()}, ' \
                  f'Train Accuracy: {train_acc_metric.result().numpy()}')
            train_loss_metric.reset_states()
            train_acc_metric.reset_states()

Start of epoch 0
Epoch: 0, Step: 0, Train Loss: 2.3380918502807617, Train Accuracy: 0.03125
Epoch: 0, Step: 20, Train Loss: 2.2421257495880127, Train Accuracy: 0.20624999701976776
Epoch: 0, Step: 40, Train Loss: 2.148804187774658, Train Accuracy: 0.2890625
Epoch: 0, Step: 60, Train Loss: 2.0579771995544434, Train Accuracy: 0.35624998807907104
Epoch: 0, Step: 80, Train Loss: 1.9493186473846436, Train Accuracy: 0.4984374940395355
Epoch: 0, Step: 100, Train Loss: 1.777862548828125, Train Accuracy: 0.48906248807907104
Epoch: 0, Step: 120, Train Loss: 1.655848503112793, Train Accuracy: 0.567187488079071
Epoch: 0, Step: 140, Train Loss: 1.5258275270462036, Train Accuracy: 0.629687488079071
Epoch: 0, Step: 160, Train Loss: 1.3608732223510742, Train Accuracy: 0.629687488079071
Epoch: 0, Step: 180, Train Loss: 1.2747933864593506, Train Accuracy: 0.643750011920929
Epoch: 0, Step: 200, Train Loss: 1.1664245128631592, Train Accuracy: 0.698437511920929
Epoch: 0, Step: 220, Train Loss: 1.05668354034

Epoch: 1, Step: 40, Train Loss: 0.47860732674598694, Train Accuracy: 0.8265625238418579
Epoch: 1, Step: 60, Train Loss: 0.4721071720123291, Train Accuracy: 0.839062511920929
Epoch: 1, Step: 80, Train Loss: 0.5611931085586548, Train Accuracy: 0.800000011920929
Epoch: 1, Step: 100, Train Loss: 0.5166772603988647, Train Accuracy: 0.7890625
Epoch: 1, Step: 120, Train Loss: 0.5618797540664673, Train Accuracy: 0.7984374761581421
Epoch: 1, Step: 140, Train Loss: 0.5035327672958374, Train Accuracy: 0.8062499761581421
Epoch: 1, Step: 160, Train Loss: 0.5068063139915466, Train Accuracy: 0.8031250238418579
Epoch: 1, Step: 180, Train Loss: 0.5001132488250732, Train Accuracy: 0.7953125238418579
Epoch: 1, Step: 200, Train Loss: 0.5135077238082886, Train Accuracy: 0.8187500238418579
Epoch: 1, Step: 220, Train Loss: 0.5542769432067871, Train Accuracy: 0.8109375238418579
Epoch: 1, Step: 240, Train Loss: 0.506097674369812, Train Accuracy: 0.8140624761581421
Epoch: 1, Step: 260, Train Loss: 0.49755248427

Epoch: 2, Step: 60, Train Loss: 0.4225797653198242, Train Accuracy: 0.854687511920929
Epoch: 2, Step: 80, Train Loss: 0.46396899223327637, Train Accuracy: 0.840624988079071
Epoch: 2, Step: 100, Train Loss: 0.38565942645072937, Train Accuracy: 0.862500011920929
Epoch: 2, Step: 120, Train Loss: 0.45579448342323303, Train Accuracy: 0.840624988079071
Epoch: 2, Step: 140, Train Loss: 0.5050408244132996, Train Accuracy: 0.824999988079071
Epoch: 2, Step: 160, Train Loss: 0.4379587173461914, Train Accuracy: 0.846875011920929
Epoch: 2, Step: 180, Train Loss: 0.4541689455509186, Train Accuracy: 0.8359375
Epoch: 2, Step: 200, Train Loss: 0.3987361490726471, Train Accuracy: 0.859375
Epoch: 2, Step: 220, Train Loss: 0.49632424116134644, Train Accuracy: 0.809374988079071
Epoch: 2, Step: 240, Train Loss: 0.4372267723083496, Train Accuracy: 0.8500000238418579
Epoch: 2, Step: 260, Train Loss: 0.40517958998680115, Train Accuracy: 0.8453124761581421
Epoch: 2, Step: 280, Train Loss: 0.4857833981513977, Tr

Epoch: 3, Step: 80, Train Loss: 0.352640837430954, Train Accuracy: 0.8890625238418579
Epoch: 3, Step: 100, Train Loss: 0.3678768575191498, Train Accuracy: 0.870312511920929
Epoch: 3, Step: 120, Train Loss: 0.43825453519821167, Train Accuracy: 0.839062511920929
Epoch: 3, Step: 140, Train Loss: 0.41054782271385193, Train Accuracy: 0.854687511920929
Epoch: 3, Step: 160, Train Loss: 0.3759442865848541, Train Accuracy: 0.859375
Epoch: 3, Step: 180, Train Loss: 0.39640331268310547, Train Accuracy: 0.8453124761581421
Epoch: 3, Step: 200, Train Loss: 0.3728145658969879, Train Accuracy: 0.8578125238418579
Epoch: 3, Step: 220, Train Loss: 0.4243638515472412, Train Accuracy: 0.846875011920929
Epoch: 3, Step: 240, Train Loss: 0.3382849097251892, Train Accuracy: 0.885937511920929
Epoch: 3, Step: 260, Train Loss: 0.4684009552001953, Train Accuracy: 0.8500000238418579
Epoch: 3, Step: 280, Train Loss: 0.41183966398239136, Train Accuracy: 0.848437488079071
Epoch: 3, Step: 300, Train Loss: 0.39059442281

Epoch: 4, Step: 100, Train Loss: 0.388611376285553, Train Accuracy: 0.859375
Epoch: 4, Step: 120, Train Loss: 0.35726362466812134, Train Accuracy: 0.8671875
Epoch: 4, Step: 140, Train Loss: 0.38609880208969116, Train Accuracy: 0.879687488079071
Epoch: 4, Step: 160, Train Loss: 0.3763195276260376, Train Accuracy: 0.8656250238418579
Epoch: 4, Step: 180, Train Loss: 0.395423948764801, Train Accuracy: 0.8421875238418579
Epoch: 4, Step: 200, Train Loss: 0.3834091126918793, Train Accuracy: 0.8578125238418579
Epoch: 4, Step: 220, Train Loss: 0.31790369749069214, Train Accuracy: 0.890625
Epoch: 4, Step: 240, Train Loss: 0.426051527261734, Train Accuracy: 0.8500000238418579
Epoch: 4, Step: 260, Train Loss: 0.36320760846138, Train Accuracy: 0.864062488079071
Epoch: 4, Step: 280, Train Loss: 0.3847706913948059, Train Accuracy: 0.859375
Epoch: 4, Step: 300, Train Loss: 0.401621013879776, Train Accuracy: 0.8515625
Epoch: 4, Step: 320, Train Loss: 0.38358768820762634, Train Accuracy: 0.8671875
Epoch

Epoch: 5, Step: 120, Train Loss: 0.3521285057067871, Train Accuracy: 0.879687488079071
Epoch: 5, Step: 140, Train Loss: 0.3624727725982666, Train Accuracy: 0.864062488079071
Epoch: 5, Step: 160, Train Loss: 0.3510926365852356, Train Accuracy: 0.879687488079071
Epoch: 5, Step: 180, Train Loss: 0.4232601225376129, Train Accuracy: 0.846875011920929
Epoch: 5, Step: 200, Train Loss: 0.3873016834259033, Train Accuracy: 0.875
Epoch: 5, Step: 220, Train Loss: 0.33781689405441284, Train Accuracy: 0.8843749761581421
Epoch: 5, Step: 240, Train Loss: 0.3341995179653168, Train Accuracy: 0.862500011920929
Epoch: 5, Step: 260, Train Loss: 0.30400174856185913, Train Accuracy: 0.895312488079071
Epoch: 5, Step: 280, Train Loss: 0.3390001654624939, Train Accuracy: 0.879687488079071
Epoch: 5, Step: 300, Train Loss: 0.3516405522823334, Train Accuracy: 0.887499988079071
Epoch: 5, Step: 320, Train Loss: 0.4212910234928131, Train Accuracy: 0.8500000238418579
Epoch: 5, Step: 340, Train Loss: 0.3551896214485168

Epoch: 6, Step: 160, Train Loss: 0.35196760296821594, Train Accuracy: 0.870312511920929
Epoch: 6, Step: 180, Train Loss: 0.3763545751571655, Train Accuracy: 0.862500011920929
Epoch: 6, Step: 200, Train Loss: 0.3407357335090637, Train Accuracy: 0.8734375238418579
Epoch: 6, Step: 220, Train Loss: 0.3463953137397766, Train Accuracy: 0.885937511920929
Epoch: 6, Step: 240, Train Loss: 0.3271496295928955, Train Accuracy: 0.8734375238418579
Epoch: 6, Step: 260, Train Loss: 0.32644709944725037, Train Accuracy: 0.871874988079071
Epoch: 6, Step: 280, Train Loss: 0.3426828384399414, Train Accuracy: 0.890625
Epoch: 6, Step: 300, Train Loss: 0.3733683228492737, Train Accuracy: 0.871874988079071
Epoch: 6, Step: 320, Train Loss: 0.29585522413253784, Train Accuracy: 0.8890625238418579
Epoch: 6, Step: 340, Train Loss: 0.35148385167121887, Train Accuracy: 0.890625
Epoch: 6, Step: 360, Train Loss: 0.2900394797325134, Train Accuracy: 0.8984375
Epoch: 6, Step: 380, Train Loss: 0.31211763620376587, Train Ac

Epoch: 7, Step: 200, Train Loss: 0.3609660565853119, Train Accuracy: 0.8671875
Epoch: 7, Step: 220, Train Loss: 0.31270498037338257, Train Accuracy: 0.8843749761581421
Epoch: 7, Step: 240, Train Loss: 0.3457261025905609, Train Accuracy: 0.8812500238418579
Epoch: 7, Step: 260, Train Loss: 0.3510914444923401, Train Accuracy: 0.8734375238418579
Epoch: 7, Step: 280, Train Loss: 0.37252795696258545, Train Accuracy: 0.862500011920929
Epoch: 7, Step: 300, Train Loss: 0.3489302098751068, Train Accuracy: 0.8828125
Epoch: 7, Step: 320, Train Loss: 0.36228078603744507, Train Accuracy: 0.8828125
Epoch: 7, Step: 340, Train Loss: 0.32854485511779785, Train Accuracy: 0.887499988079071
Epoch: 7, Step: 360, Train Loss: 0.260001540184021, Train Accuracy: 0.9078124761581421
Epoch: 7, Step: 380, Train Loss: 0.32857879996299744, Train Accuracy: 0.8765624761581421
Epoch: 7, Step: 400, Train Loss: 0.33740636706352234, Train Accuracy: 0.8890625238418579
Epoch: 7, Step: 420, Train Loss: 0.36091744899749756, Tr

Epoch: 8, Step: 220, Train Loss: 0.3440362215042114, Train Accuracy: 0.8734375238418579
Epoch: 8, Step: 240, Train Loss: 0.3526243269443512, Train Accuracy: 0.8671875
Epoch: 8, Step: 260, Train Loss: 0.2744380831718445, Train Accuracy: 0.8968750238418579
Epoch: 8, Step: 280, Train Loss: 0.27668482065200806, Train Accuracy: 0.90625
Epoch: 8, Step: 300, Train Loss: 0.3515588343143463, Train Accuracy: 0.879687488079071
Epoch: 8, Step: 320, Train Loss: 0.33447834849357605, Train Accuracy: 0.895312488079071
Epoch: 8, Step: 340, Train Loss: 0.29290783405303955, Train Accuracy: 0.893750011920929
Epoch: 8, Step: 360, Train Loss: 0.29457104206085205, Train Accuracy: 0.893750011920929
Epoch: 8, Step: 380, Train Loss: 0.29704731702804565, Train Accuracy: 0.8843749761581421
Epoch: 8, Step: 400, Train Loss: 0.2926050126552582, Train Accuracy: 0.879687488079071
Epoch: 8, Step: 420, Train Loss: 0.34505701065063477, Train Accuracy: 0.8828125
Epoch: 8, Step: 440, Train Loss: 0.2771774232387543, Train A

Epoch: 9, Step: 240, Train Loss: 0.2994652986526489, Train Accuracy: 0.8984375
Epoch: 9, Step: 260, Train Loss: 0.2968166470527649, Train Accuracy: 0.901562511920929
Epoch: 9, Step: 280, Train Loss: 0.26736241579055786, Train Accuracy: 0.8999999761581421
Epoch: 9, Step: 300, Train Loss: 0.3101779520511627, Train Accuracy: 0.8812500238418579
Epoch: 9, Step: 320, Train Loss: 0.31796202063560486, Train Accuracy: 0.8890625238418579
Epoch: 9, Step: 340, Train Loss: 0.30111825466156006, Train Accuracy: 0.885937511920929
Epoch: 9, Step: 360, Train Loss: 0.30865055322647095, Train Accuracy: 0.893750011920929
Epoch: 9, Step: 380, Train Loss: 0.2427304983139038, Train Accuracy: 0.921875
Epoch: 9, Step: 400, Train Loss: 0.2889936566352844, Train Accuracy: 0.901562511920929
Epoch: 9, Step: 420, Train Loss: 0.29421791434288025, Train Accuracy: 0.890625
Epoch: 9, Step: 440, Train Loss: 0.3203403353691101, Train Accuracy: 0.885937511920929
Epoch: 9, Step: 460, Train Loss: 0.3228341042995453, Train Ac

In [46]:
# do test

@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=True)
    loss = loss_object(targets, predictions)
    test_loss_metric(loss)
    test_acc_metric(targets, predictions)


for inputs, targets in test_ds:
    test_step(inputs, targets)

print(f'Test Loss: {test_loss_metric.result().numpy()} Test Accuracy: {test_acc_metric.result().numpy()}')
test_loss_metric.reset_states()
test_acc_metric.reset_states()

Test Loss: 0.32756760716438293 Test Accuracy: 0.883400022983551
